In [1]:
# from https://www.kaggle.com/users/123235/fchollet/digit-recognizer/simple-deep-mlp-with-keras
%pylab inline

import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.utils import np_utils
from keras.layers.core import Dense, Activation, Dropout

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Read data
train = pd.read_csv('../data/train.csv')
labels = train.ix[:,0].values.astype('int32')
X_train = (train.ix[:,1:].values).astype('float32')
X_test = (pd.read_csv('../data/test.csv').values).astype('float32')

In [3]:
# convert list of labels to binary class matrix
y_train = np_utils.to_categorical(labels) 

In [4]:
# pre-processing: divide by max and substract mean
scale = np.max(X_train)
X_train /= scale
X_test /= scale

mean = np.std(X_train)
X_train -= mean
X_test -= mean

input_dim = X_train.shape[1]
nb_classes = y_train.shape[1]

In [5]:
# Here's a Deep Dumb MLP (DDMLP)
model = Sequential()
model.add(Dense(input_dim, 128, init='lecun_uniform'))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(128, 128, init='lecun_uniform'))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(128, nb_classes, init='lecun_uniform'))
model.add(Activation('relu'))

In [6]:
# we'll use MSE (mean squared error) for the loss, and RMSprop as the optimizer
model.compile(loss='mse', optimizer='rmsprop')

In [7]:
print("Training...")
model.fit(X_train, y_train, nb_epoch=10, batch_size=16, validation_split=0.1, show_accuracy=True, verbose=2)

Training...
Train on 37800 samples, validate on 4200 samples
Epoch 0
10s - loss: 0.0193 - acc.: 0.9045 - val. loss: 0.0119 - val. acc.: 0.9362
Epoch 1
12s - loss: 0.0121 - acc.: 0.9431 - val. loss: 0.0079 - val. acc.: 0.9531
Epoch 2
12s - loss: 0.0094 - acc.: 0.9526 - val. loss: 0.0074 - val. acc.: 0.9569
Epoch 3
10s - loss: 0.0077 - acc.: 0.9608 - val. loss: 0.0061 - val. acc.: 0.9619
Epoch 4
13s - loss: 0.0072 - acc.: 0.9619 - val. loss: 0.0055 - val. acc.: 0.9664
Epoch 5
10s - loss: 0.0064 - acc.: 0.9659 - val. loss: 0.0054 - val. acc.: 0.9660
Epoch 6
11s - loss: 0.0057 - acc.: 0.9694 - val. loss: 0.0055 - val. acc.: 0.9652
Epoch 7
11s - loss: 0.0052 - acc.: 0.9720 - val. loss: 0.0049 - val. acc.: 0.9681
Epoch 8
10s - loss: 0.0051 - acc.: 0.9727 - val. loss: 0.0053 - val. acc.: 0.9671
Epoch 9
12s - loss: 0.0047 - acc.: 0.9750 - val. loss: 0.0050 - val. acc.: 0.9683


In [8]:
print("Generating test predictions...")
preds = model.predict_classes(X_test, verbose=0)

Generating test predictions...


In [9]:
def write_preds(preds, fname):
    pd.DataFrame({"ImageId": list(range(1,len(preds)+1)), "Label": preds}).to_csv(fname, index=False, header=True)

write_preds(preds, "keras-mlp.csv")